# Final Training and Model Export

In [ ]:
import pandas as pd
import numpy as np
import catboost as cat
import pickle
import os
SEED = 42

In [ ]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 10000):
        display(df)

In [ ]:
FINAL_FEATURES_PATH = 'processed_data/final_features.feather'
FINAL_MODEL_SAVE_PATH = 'models/final_model.cbm'

# Replace with your best parameters from Optuna tuning
BEST_PARAMS = {
    'iterations': 1000,
    'learning_rate': 0.08579514753476217, 
    'depth': 8,           
    'l2_leaf_reg': 1.1367839827002824,   
    # Add other best parameters found, excluding fit-specific ones like early_stopping_rounds
    'loss_function': 'MultiClass',
    'eval_metric': 'MultiClass',
    'random_seed': 42,
    'verbose': 100, # Show progress
    'verbose': False,
    'allow_writing_files': False,
}

In [ ]:
features_df_final = pd.read_feather(FINAL_FEATURES_PATH)
# features_df_final = pd.read_csv(FINAL_FEATURES_PATH)
print(f"Features loaded. Shape: {features_df_final.shape}")

In [ ]:
required_cols = ['sequence_id', 'subject', 'gesture', 'gesture_encoded']
feature_columns = [col for col in features_df_final.columns if col not in required_cols]

X_final = features_df_final[feature_columns]
y_final = features_df_final['gesture_encoded']

In [ ]:
print(f"Final training data prepared:")
print(f" - X_final shape: {X_final.shape}")
print(f" - y_final shape: {y_final.shape}")
print(f" - Unique classes: {y_final.nunique()}")

In [ ]:
final_model = cat.CatBoostClassifier(**BEST_PARAMS)

In [ ]:
final_model.fit(X_final, y_final)

In [ ]:
# --- Save the Trained Model ---
print(f"\nSaving final model to: {FINAL_MODEL_SAVE_PATH}")
try:
    # Save in CatBoost's native format for easy loading later
    final_model.save_model(FINAL_MODEL_SAVE_PATH)
    print("Final model saved successfully.")
except Exception as e:
    print(f"Error saving final model: {e}")
    raise

# --- Optional: Save feature names for use in submission script ---
# This ensures the submission script uses the exact same feature order.
#feature_names_path = 'final_feature_names.pkl'
#try:
#    with open(feature_names_path, 'wb') as f:
#        pickle.dump(feature_columns, f)
#    print(f"Feature names saved to: {feature_names_path}")
#except Exception as e:
#    print(f"Error saving feature names: {e}")